# 🧪 Anomaly Scoring & Heatmaps

In this step, we transform ViT-based features into meaningful **anomaly scores** and **visual heatmaps**.

Using the patch-level embeddings from *test images*, we compare them against the **feature bank** built from `train/good` data. This allows us to detect both image-level anomalies and localize defective regions.

---

### Goals of this notebook:

- Compute **image-level anomaly scores** using:
  - **KNN distance** (via FAISS)
  - Optionally: **Mahalanobis distance**
- Generate **pixel-level heatmaps** by projecting patch distances to spatial grids
- Upsample and visualize heatmaps as overlays on the original images
- Determine an optimal threshold for binary decisions (Youden’s J or best F1)
- Evaluate detection performance using standard metrics (AUROC, PRO, etc.)

---

This scoring approach works **fully unsupervised**, relying only on normal (`train/good`) examples.

By the end of this notebook, you’ll be able to:
- Score all test images
- Visualize and interpret heatmaps
- Quantify anomaly detection performance


## 1️⃣ Load Features & Metadata

In this section, we load the precomputed ViT embeddings and metadata generated during the feature extraction step.

Specifically, we will:
- Load `.npz` feature files (CLS + patch embeddings) for each category
- Load the corresponding `.csv` metadata files
- Merge them into a single DataFrame for easy access and filtering

This unified structure will allow us to:
- Quickly isolate `train/good` features (for scoring)
- Select `test` images for evaluation
- Link embeddings back to image paths for visualization

---

Each entry contains:
- `cls`: global image embedding (ViT CLS token)
- `patches`: local patch embeddings
- `patch_hw`: patch grid shape (e.g., 16×16)
- metadata (path, label, split, etc.)


In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
import pickle

BASE_DIR = Path("..").resolve()

CFG = {
    "dino": {
        "FEAT_DIR": BASE_DIR / "features_dinov2_b14",
        "BANK_DIR": BASE_DIR / "featurebanks" / "dinov2_b14",
        "FEAT_NPZ_TPL": "{cat}_dinov2_vitb14.npz",
        "BANK_NPZ_TPL": "{cat}_bank_dinov2_b14.npz",
    },
    "mae": {
        "FEAT_DIR": BASE_DIR / "features_mae_b16",
        "BANK_DIR": BASE_DIR / "featurebanks" / "mae_b16",
        "FEAT_NPZ_TPL": "{cat}_mae_b16.npz",
        "BANK_NPZ_TPL": "{cat}_bank_mae_b16.npz",
    },
}

def categories_from_feat_dir(feat_dir: Path):
    cats = []
    for p in feat_dir.glob("*_meta.csv"):
        name = p.name
        if name.endswith("_meta.csv"):
            cats.append(name[:-len("_meta.csv")])
    return sorted(set(cats))

def load_feature_data(feat_dir: Path, feat_tpl: str, category: str):
    feat_file = feat_dir / feat_tpl.format(cat=category)
    meta_file = feat_dir / f"{category}_meta.csv"
    data = np.load(feat_file)
    df = pd.read_csv(meta_file)
    return {
        "patches": data["patches"],
        "cls": data.get("cls"),
        "patch_hw": tuple(data["patch_hw"]),
        "meta": df
    }

def load_feature_bank(bank_dir: Path, bank_tpl: str, category: str):
    bank_file = bank_dir / bank_tpl.format(cat=category)
    meta_file = bank_dir / f"{category}_bank_meta.csv"
    data = np.load(bank_file)
    df = pd.read_csv(meta_file)
    return {
        "patches": data["patches"],
        "patch_hw": tuple(data["patch_hw"]),
        "meta": df
    }

def save_dicts_to_disk(backbone: str, features_all: dict, banks: dict, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    for cat in features_all:
        with open(out_dir / f"{cat}_features.pkl", "wb") as f:
            pickle.dump(features_all[cat], f)
        with open(out_dir / f"{cat}_bank.pkl", "wb") as f:
            pickle.dump(banks[cat], f)
        print(f"✅ Saved {cat} ({backbone}) to {out_dir}")

def load_backbone(name: str):
    cfg = CFG[name]
    feat_dir, bank_dir = cfg["FEAT_DIR"], cfg["BANK_DIR"]
    feat_tpl, bank_tpl = cfg["FEAT_NPZ_TPL"], cfg["BANK_NPZ_TPL"]

    categories = categories_from_feat_dir(feat_dir)
    features_all, banks = {}, {}
    for cat in categories:
        features_all[cat] = load_feature_data(feat_dir, feat_tpl, cat)
        banks[cat] = load_feature_bank(bank_dir, bank_tpl, cat)
    return categories, features_all, banks

# Load and save DINO
cats_dino, features_all_dino, banks_dino = load_backbone("dino")
save_dicts_to_disk("dino", features_all_dino, banks_dino, BASE_DIR / "cached_dicts" / "dino")

# Load and save MAE
cats_mae, features_all_mae, banks_mae = load_backbone("mae")
save_dicts_to_disk("mae", features_all_mae, banks_mae, BASE_DIR / "cached_dicts" / "mae")




✅ Saved bottle (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved cable (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved capsule (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved carpet (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved grid (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved hazelnut (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved leather (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved metal_nut (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved pill (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved screw (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved tile (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved toothbrush (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved transistor (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved wood (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved zipper (dino) to C:\Users\Fredi\MVTec\cached_dicts\dino
✅ Saved bottle (mae) to C:\Users\Fr

### Data Structures

After loading, we store the results in two dictionaries.  
This design separates **all extracted embeddings** from the **reference banks**,  
making evaluation, visualization, and anomaly scoring more convenient.

---

- **`features_all`**  
  Contains the **full embeddings** (train + test) for each category.  
  Each entry is a dictionary with:
  - `cls`: global image embeddings `[N, D]`
  - `patches`: patch embeddings `[N, P, D]`
  - `patch_hw`: patch grid dimensions `(H, W)`
  - `meta`: metadata DataFrame (paths, labels, splits)

  → Used when we need to compare across *all images* (e.g., evaluation, visualization).

---

- **`banks`**  
  Contains only the **feature bank**: patch embeddings from `train/good` images.  
  Each entry is a dictionary with:
  - `patches`: patch embeddings `[N, P, D]`
  - `patch_hw`: patch grid dimensions `(H, W)`
  - `meta`: metadata DataFrame for good samples

  → Used as the **reference set** for anomaly scoring: test patches are compared against this bank.


These structures allow us to seamlessly switch between global evaluation (`features_all`) and reference-based scoring (`banks`). In the next step, we leverage them to compute image-level anomaly scores and pixel-level heatmaps.




## 2️⃣ Build Search Index / Distribution

After extracting features, the next step is to organize the 
**reference distribution of normal data**. This serves as the 
foundation for anomaly detection: test samples are compared 
against this reference to estimate how "normal" or "abnormal" they are.

Two complementary approaches are considered:

- **KNN-based feature banks:**  
  Patch embeddings from `train/good` images are stored in a 
  nearest-neighbor index (e.g. FAISS).  
  → At inference, each test patch is compared to its closest neighbors 
  in this bank.

- **Mahalanobis distribution:**  
  A parametric approach where we compute the mean vector and covariance 
  matrix of `train/good` patches per category.  
  → Anomaly scores are then derived from the multivariate distance 
  between test patches and this distribution.

This step transforms the raw feature embeddings into 
a **searchable structure** (index or distribution) that enables 
efficient and principled anomaly scoring in the following step.

